# The Structure of Indices of Heart Rate Variability
## Introduction
### Context
The recording of cardiovascular activity with the help of electrocardiographic (ECG) and photoplethysmographic (PPG) signals provides a cost-efficient and reliable window into the autonomic nervous system (ANS). These biosignals provide complex patterns of variability present in any biological systems, i.e. Heart-Rate Variability (HRV). These patterns exhibit the underlying organization of ANS and can be described as mathematical chaos (Shaffer et al. 2015). The aim of this study is to explore the factorial structure of the patterns of HRV.
### Definition
HRV can be defined as the oscillation of the interval between consecutive heartbeats, or inter-beat intervals (IBIs), as regulated by the nervous system. The heart is dually innervated by the two constituents of the autonomic nervous system (ANS) namely, the sympathetic (SNS) and parasympathetic (PNS) nervous system. The ANS dynamically regulates an individual’s heart rate in response to internal, and external events, e.g.  an increase in blood pressure related or not to a stressful event. While the PNS exert more dynamic control over heart rate, the SNS is reported to have more influence on myocardial contractility, meaning they have distinct modes of innervation (Berntson, Quigley, Norman, & Lozano, 2017, p. 189). The PNS changes HR in terms of miliseconds, while the SNS acts on HR in terms of seconds. In this sense, one can suppose that the factorial structure of HRV's indices is informative of autonomic organization.
### Research problem
Although HRV is conceptually simple, the relations between its multiple indices still remain complex to interpret. We use the term indices to describe each variable derived from the IBIs distribution and the  distribution of differences between successive IBIs. The factorial structure of these variables is prone to be altered according to the experimental condition, but also the context, body position, metabolism etc. Significant work has been done to provide a clear methodological framework in response to these issues. Researchers have provided guidelines to validate and share results transparently (Quintana &. Heathers, 2014; Quintana et al. 2016), on top of classic standards (Task Force, 1996), and clear systematic definition of HRV variables (Shaffer et al. 2015). However, Laborde, Mosley & Thayer (2017) argued that some confusion still is present as to the relationship between variables computed from IBIs and their meaning in a given experimental context, i.e. what’s the level of cardiac vagal tone. We aim to respond to this issue by discussing the statistical relationships of these variables independently of experimental context. A principal component analysis (PCA) is used to reduce the dimensionality of our data and isolate factors that could be related to ANS' components contribution. 
## Method
In light of the recommendations published in the last decade, this study explores the statistical relationships between all indices of HRV using open-access databases of annotated ECG signals and NeuroKit data preparation pipeline. In order to allow proper isolation of non-redundant related indices of HRV, PCA will be used as a dimensionality reduction technique to explore their relationships.

#### Dependencies

In [1]:
import pandas as pd
import numpy as np
import neurokit2 as nk

In [3]:
pwd

'C:\\Users\\sangfrois\\NeuroKit\\studies\\hrv_structure'

### Databases
Procedure to dowload the data is available through [...](). Annotated R-peaks are used to compute HRV.

In [2]:
# Load True R-peaks location
files_list = [pd.read_csv("../../data/gudb/Rpeaks.csv"),
             pd.read_csv("../../data/mit_arrhythmia/Rpeaks.csv"),
             pd.read_csv("../../data/mit_normal/Rpeaks.csv"),
             pd.read_csv("../../data/fantasia/Rpeaks.csv")]

FileNotFoundError: [Errno 2] File ../../data/gudb/Rpeaks.csv does not exist: '../../data/gudb/Rpeaks.csv'

### Glasgow University Database

The GUDB Database (Howell & Porr, 2018) contains ECGs from 25 subjects.
Each subject was recorded performing 5 different tasks for two minutes
(sitting, doing a maths test on a tablet, walking on a treadmill,
running on a treadmill, using a hand bike). The sampling rate is 250Hz
for all the conditions.

The script to download and format the database using the
[**ECG-GUDB**](https://github.com/berndporr/ECG-GUDB) Python package by
Bernd Porr can be found
[**here**](https://github.com/neuropsychology/NeuroKit/blob/dev/data/gudb/download_gudb.py).

### MIT-BIH Arrhythmia Database

The MIT-BIH Arrhythmia Database (MIT-Arrhythmia; Moody & Mark, 2001)
contains 48 excerpts of 30-min of two-channel ambulatory ECG recordings
sampled at 360Hz and 25 additional recordings from the same participants
including common but clinically significant arrhythmias (denoted as the
`MIT-Arrhythmia-x` database).

The script to download and format the database using the can be found
[**here**](https://github.com/neuropsychology/NeuroKit/blob/dev/data/mit_arrhythmia/download_mit_arrhythmia.py).

<!-- ### MIT-BIH Noise Stress Test Database -->

<!-- The MIT-BIH Noise Stress Test Database [MIT-NST; @moody1984bih] features two 30-minute recordings distorted by adding different types and levels of synthesized noise typical of electrode motion artefacts. -->

### MIT-BIH Normal Sinus Rhythm Database

This database includes 18 clean long-term ECG recordings of subjects.
Due to memory limits, we only kept the second hour of recording of each
participant.

The script to download and format the database using the can be found
[**here**](https://github.com/neuropsychology/NeuroKit/blob/dev/data/mit_normal/download_mit_normal.py).

### Lobachevsky University Electrocardiography Database

The Lobachevsky University Electrocardiography Database (LUDB;
Kalyakulina et al., 2018) consists of 200 10-second 12-lead ECG signal
records representing different morphologies of the ECG signal. The ECGs
were collected from healthy volunteers and patients, which had various
cardiovascular diseases. The boundaries of P, T waves and QRS complexes
were manually annotated by cardiologists for all 200 records.

### Fantasia Database

The Fantasia database (Iyengar, Peng, Morin, Goldberger, & Lipsitz,
1996) consists of twenty young and twenty elderly healthy subjects. All
subjects remained in a resting state in sinus rhythm while watching the
movie Fantasia (Disney, 1940) to help maintain wakefulness. The
continuous ECG signals were digitized at 250 Hz. Each heartbeat was
annotated using an automated arrhythmia detection algorithm, and each
beat annotation was verified by visual inspection.


## Procedure

### NeuroKit Data preparation

In [ ]:
# Get results
all_results = pd.DataFrame()

# Get to a participant's file inside a given database
for file in files_list:
    for database in np.unique(file["Database"]):
        data = file[file["Database"] == database]
        
        # Get sample rate and millisecond interval for each participant 
        for participant in np.unique(data["Participant"]):
            data_participant = data[data["Participant"] == participant]
            sampling_rate = np.unique(data_participant["Sampling_Rate"])[0]
            rpeaks = data_participant["Rpeaks"].values
            
            # Compute HRV indices of a given participant
            results = nk.hrv(rpeaks, sampling_rate=sampling_rate)
            results["Participant"] = participant
            results["Database"] = database
            results["Recording_Length"] = rpeaks[-1] / sampling_rate / 60
            
            # keep results
            all_results = pd.concat([all_results, results], axis=0)

# push results in csv format
all_results.to_csv("data.csv", index=False)

### Time
Contains time domain HRV metrics:
   - **RMSSD**: The square root of the mean of the sum of successive differences between
        adjacent RR intervals. It is equivalent (although on another scale) to SD1, and
        therefore it is redundant to report correlations with both (Ciccone, 2017).
   - **MeanNN**: The mean of the RR intervals.
   - **SDNN**: The standard deviation of the RR intervals.
   - **SDSD**: The standard deviation of the successive differences between RR intervals.
   - **CVNN**: The standard deviation of the RR intervals (SDNN) divided by the mean of the RR
        intervals (MeanNN).
   - **CVSD**: The root mean square of the sum of successive differences (RMSSD) divided by the
        mean of the RR intervals (MeanNN).
   - **MedianNN**: The median of the absolute values of the successive differences between RR intervals.
   - **MadNN**: The median absolute deviation of the RR intervals.
   - **HCVNN**: The median absolute deviation of the RR intervals (MadNN) divided by the median
        of the absolute differences of their successive differences (MedianNN).
   - **IQRNN**: The interquartile range (IQR) of the RR intervals.
   - **pNN50**: The proportion of RR intervals greater than 50ms, out of the total number of RR intervals.
   - **pNN20**: The proportion of RR intervals greater than 20ms, out of the total number of RR intervals.
   - **TINN**: A geometrical parameter of the HRV, or more specifically, the baseline width of
        the RR intervals distribution obtained by triangular interpolation, where the error of least
        squares determines the triangle. It is an approximation of the RR interval distribution.
   - **HTI**: The HRV triangular index, measuring the total number of RR intervals divded by the
        height of the RR intervals histogram.

### Frequency
   - **ULF**: The spectral power density pertaining to ultra low frequency band i.e., .0 to .0033 Hz
        by default.
   - **VLF**: The spectral power density pertaining to very low frequency band i.e., .0033 to .04 Hz
        by default.
   - **LF**: The spectral power density pertaining to low frequency band i.e., .04 to .15 Hz by default.
   - **HF**: The spectral power density pertaining to high frequency band i.e., .15 to .4 Hz by default.
   - **VHF**: The variability, or signal power, in very high frequency i.e., .4 to .5 Hz by default.
   - **LFn**: The normalized low frequency, obtained by dividing the low frequency power by
        the total power.
   - **HFn**: The normalized high frequency, obtained by dividing the low frequency power by
        the total power.
   - **LnHF**: The log transformed HF.

### Non-Linear
   - **SD1**: SD1 is a measure of the spread of RR intervals on the Poincaré plotperpendicular to the line of identity. It is an index of short-term RR interval fluctuations, i.e., beat-to-beat variability. It is equivalent (although on another scale) to RMSSD, and therefore it is redundant to report correlations with both (Ciccone, 2017).

   - **SD2**: SD2 is a measure of the spread of RR intervals on the Poincaré plot along the line of identity. It is an index of long-term RR interval fluctuations.

   - **SD1SD2**: the ratio between short and long term fluctuations of the RR intervals (SD1 divided by SD2).

   - **S**: Area of ellipse described by SD1 and SD2 (``pi * SD1 * SD2``). It is proportional to *SD1SD2*.

   - **CSI**: The Cardiac Sympathetic Index (Toichi, 1997), calculated by dividing the longitudinal variability of the Poincaré plot (``4*SD2``) by its transverse variability (``4*SD1``).

   - **CVI**: The Cardiac Vagal Index (Toichi, 1997), equal to the logarithm of the product of longitudinal (``4*SD2``) and transverse variability (``4*SD1``).

   - **CSI_Modified**: The modified CSI (Jeppesen, 2014) obtained by dividing the square of the longitudinal variability by its transverse variability.

**Indices of Heart Rate Asymmetry (HRA), i.e., asymmetry of the Poincaré plot** (Yan, 2017):

   - **GI**: Guzik's Index, defined as the distance of points above line of identity (LI) to LI divided by the distance of all points in Poincaré plot to LI except those that are located on LI.

   - **SI**: Slope Index, defined as the phase angle of points above LI divided by the phase angle of all points in Poincaré plot except those that are located on LI.

   - **AI**: Area Index, defined as the cumulative area of the sectors corresponding to the points that are located above LI divided by the cumulative area of sectors corresponding to all points in the Poincaré plot except those that are located on LI.

   - **PI**: Porta's Index, defined as the number of points below LI divided by the total number of points in Poincaré plot except those that are located on LI.

   - **SD1d** and **SD1a**: short-term variance of contributions of decelerations (prolongations of RR intervals) and accelerations (shortenings of RR intervals), respectively (Piskorski,  2011).

   - **C1d** and **C1a**: the contributions of heart rate decelerations and accelerations to short-term HRV, respectively (Piskorski,  2011).

   - **SD2d** and **SD2a**: long-term variance of contributions of decelerations (prolongations of RR intervals) and accelerations (shortenings of RR intervals), respectively (Piskorski,  2011).

   - **C2d** and **C2a**: the contributions of heart rate decelerations and accelerations to long-term HRV, respectively (Piskorski,  2011).

   - **SDNNd** and **SDNNa**: total variance of contributions of decelerations(prolongations of RR intervals) and accelerations (shortenings of RR intervals), respectively (Piskorski,  2011).

   - **Cd** and **Ca**: the total contributions of heart rate decelerations and  accelerations to HRV.

**Indices of Heart Rate Fragmentation** (Costa, 2017):

   - **PIP**: Percentage of inflection points of the RR intervals series.

   - **IALS**: Inverse of the average length of the acceleration/deceleration segments.

   - **PSS**: Percentage of short segments.

   - **PAS**: IPercentage of NN intervals in alternation segments.

**Indices of Complexity**:

   - **ApEn**: The approximate entropy measure of HRV, calculated by `entropy_approximate()`.

   - **SampEn**: The sample entropy measure of HRV, calculated by `entropy_sample()`

# References
Ali, S. S., Lifshitz, M., & Raz, A. (2014). Empirical neuroenchantment: from reading minds to thinking critically. Frontiers in human neuroscience, 8, 357. https://doi.org/10.3389/fnhum.2014.00357